In [1]:
import os
import pickle
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
with open("out.pickle", "rb") as fp:
    data = pickle.load(fp)

In [3]:
px.scatter(x=data.keys(), y=data.values())

In [4]:
X = np.array(list(data.values()), dtype=np.int64)
dX = np.diff(X)
ddX = np.diff(dX)
t = list(data.keys())
px.scatter(x=t[1:], y=dX)

We can see from the diff-plot that there is some periodicity to the graph. As a brute-force, we just observe the period and start of the period normally. This is done at the points t = 193 + 1 and repeats again at t = 324 + 1

In [5]:
start = 193 + 1
period = 324 +1 - start
print(f"Period: {period}")

Period: 131


In [6]:
offset = 0
print(np.diff(dX[start - 1 + offset::period]))
px.scatter(x=t[start + offset::period], y=dX[start - 1 + offset::period])

[331 331 331 331 331 331]


We can see that the diff increases with a constant number (in this case, 331. Let's assume that the original diff-value, and the increase in the value, is different for all time steps within a period, and let's calculate them).

In [7]:
orig_diff = []
for i in range(period):
    orig_diff.append(dX[start - 1 + i])

increases = []
for i in range(period):
    increases.append(dX[start - 1 + period + i] - orig_diff[i])

Now let's see if we can create a formula for calculating the original curve.

In [8]:
t = 194
x = int(X[t - 1])
idx = 0
calculated = {t: x}
while t < 950:
    t += 1
    x += orig_diff[idx]
    orig_diff[idx] += increases[idx]
    calculated[t] = x
    idx = (idx + 1) % period


In [9]:
fig = px.scatter(x=data.keys(), y=data.values())
fig.add_scatter(x=list(calculated.keys()), y=list(calculated.values()))
fig.update_layout(xaxis=dict(range=[190, 200]), yaxis=dict(range=[20000, 40000]))

In [10]:
diff = [data[i] - calculated[i] for i in calculated.keys()]
px.scatter(x = list(calculated.keys()),y=diff)

We can see that this formula works wonderfully, so let's extend it out to the given number

In [11]:
while t < 26501365:
    t += 1
    x += orig_diff[idx]
    orig_diff[idx] += increases[idx]
    calculated[t] = x
    idx = (idx + 1) % period

In [15]:
print(f"The value at t={t}: x={x}")


The value at t=26501365: x=607334325965751
